In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())


CUDA available: False


In [3]:
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!pip install -q langchain
!pip install -q langchain-community
!pip install -q langchain-text-splitters
!pip install -q python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 3.6 MB/s eta 0:00:00


In [4]:
import unicodedata

from docx import Document as DocxDocument
from langchain_core.documents import Document

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

print("All imports successful ")


All imports successful 


In [5]:
from google.colab import files
uploaded = files.upload()

print("Uploaded files:", list(uploaded.keys()))


Saving Rag-docs.docx to Rag-docs.docx
Uploaded files: ['Rag-docs.docx']


In [6]:
documents = []

for filename in uploaded.keys():
    if filename.lower().endswith(".docx"):
        doc = DocxDocument(filename)

        text = "\n".join(
            [p.text for p in doc.paragraphs if p.text.strip()]
        )

        text = unicodedata.normalize("NFC", text)

        documents.append(
            Document(
                page_content=text,
                metadata={"source": filename}
            )
        )

print("Total documents loaded:", len(documents))
print("Sample text:\n", documents[0].page_content[:400])


Total documents loaded: 1
Sample text:
 मूर्खभृत्यस्य
"अरे शंखनाद, गच्छापणम्, शर्कराम् आनय ।" इति स्वभृत्यम् शंखनादम् गोवर्धनदासः आदिशति । ततः शंखनादः आपणम् गच्छति, शर्कराम् जीर्णे वस्त्रे न्यस्यति च । तस्मात् जीर्णवस्त्रात् मार्गे एव सर्वापि शर्करा स्त्रवति । ततः गोवर्धनदासः कोपेन शंखनादम् वदति, "अरे मूढ, कुत्रास्ति शर्करा ? शर्करादिकम् एवम् जीर्णेन वस्त्रेण न एवानयन्ति कदापि । इतःपरम् किमपि वस्तुजातम् दृढायाम् सन्चिकायाम् निक्षिप्य आन


In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=80
)

chunks = text_splitter.split_documents(documents)

print("Total chunks created:", len(chunks))


Total chunks created: 31


In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings


In [11]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={"device": "cpu"}
)


/tmp/ipython-input-2833012921.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [12]:
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

print("FAISS vector store built successfully ")


FAISS vector store built successfully 


In [13]:
print(len(vectorstore.index_to_docstore_id))


31


In [27]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [28]:
def build_prompt(context, question):
    return f"""
You are a Sanskrit scholar.

Answer the question using the context below.
Answer in Sanskrit.
If unsure, give the closest possible answer from the text.

Context:
{context}

Question:
{question}

Answer:
"""


In [29]:
def ask_rag(question):
    docs = retriever.invoke(question)
    context = "\n".join([d.page_content for d in docs])

    prompt = build_prompt(context, question)

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(
      **inputs,
      max_new_tokens=150,
      min_new_tokens=10,
      do_sample=False
)

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


In [30]:
question = "शंखनादः किम् कृतवान् ?"
print(ask_rag(question))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<extra_id_0> । Context: कथनः कथनः


In [31]:
question = "घण्टाकर्णः कथं नष्टः अभवत् ?"
print(ask_rag(question))


<extra_id_0> । Context: कथाः कथाः


In [32]:
import time

start = time.time()
print(ask_rag("कालीदासः किमर्थं चतुरः आसीत् ?"))
end = time.time()

print("Latency (seconds):", round(end - start, 2))


<extra_id_0> ? ? ? ? 
Latency (seconds): 1.47
